In [1]:
import sys,os
import csv
import pandas as pd

In [2]:
files = ['flex_data_2-6.xlsx', 
        'flex_data_6-9.xlsx',
        'scc_data_1-6.xlsx',
        'scc_data_6-9.xlsx']

all_sn_data = pd.DataFrame()
for idx, val in enumerate(files):
    file = val
    print("Reading file ", file)
    data = pd.read_excel(file)
    #data.info()
    for idx, row in data.iterrows():
        row['MotherSN'] = row['MotherSN'].strip("\'")
    all_sn_data = all_sn_data.append(data)
    
all_sn_data.info()

Reading file  flex_data_2-6.xlsx
Reading file  flex_data_6-9.xlsx
Reading file  scc_data_1-6.xlsx
Reading file  scc_data_6-9.xlsx
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97539 entries, 0 to 25286
Data columns (total 2 columns):
MotherSN    97539 non-null object
ChildSN     97486 non-null object
dtypes: object(2)
memory usage: 2.2+ MB


In [3]:
out_columns = ['PackDate', 'MotherSN', 'MB', 'NIBP', 'SPO2', 'GLU', 'ECG', 'NFC', 'INTERFACE', 'BATTERY', 'LCD', '3G']
all_mother_sn_data = pd.read_excel("ProductSN_3-9.xlsx")
all_mother_sn_data.info()
all_mother_sn_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8647 entries, 0 to 8646
Data columns (total 2 columns):
ProductSN    8647 non-null object
PackDate     8647 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 135.2+ KB


,ProductSN,PackDate
0,GT10006244181W0053,2018-03-05
1,GT1000624418210035,2018-03-05
2,GT1000624418210117,2018-03-05
3,GT1000624418210123,2018-03-05
4,GT1000624418210138,2018-03-05


In [13]:
out_data = pd.DataFrame(columns=out_columns)
for i, mother in all_mother_sn_data.iterrows():
    mother_sn = mother['ProductSN'][2:]
    child_data = all_sn_data[all_sn_data['MotherSN'] == mother_sn]
    product_sn = pd.Series([mother['PackDate'], mother_sn, None, None, None, None, None, None, None, None, None, None], 
                           out_columns)
    
    for idx, row in child_data.iterrows():
        child_sn = str(row['ChildSN'])
        
        if (child_sn[:3] == '000'): 
            product_sn['MB'] = child_sn[:16]
        elif (child_sn[:4] == '0400'):
            product_sn['GLU'] = child_sn[:16]
        elif (child_sn[:4] == '2000' or child_sn[:4] == '1200'):
            product_sn['NIBP'] = child_sn[:16]
        elif (child_sn[:4] == '1100'):
            product_sn['SPO2'] = child_sn[:16]
        elif (child_sn[:4] == 'SZGR' or child_sn[:4] == 'JSGR'):
            product_sn['ECG'] = child_sn[:13]
        elif(child_sn[:4] == '0900'):
            product_sn['NFC'] = child_sn[:16]
        elif(child_sn[:4] == '1500'):
            product_sn['INTERFACE'] = child_sn[:16]
        elif(child_sn[:4] == '4555'):
            product_sn['BATTERY'] = child_sn[:15]
        elif(child_sn[:2] == 'XH'):
            product_sn['BATTERY'] = child_sn[:11]
        elif(child_sn[:4] == 'XR-X'):
            product_sn['LCD'] = child_sn[:21]
        elif(child_sn[:3] == 'A00'):
            product_sn['3G'] = child_sn

    out_data = out_data.append(product_sn, ignore_index=True)
    

In [14]:
out_data.info()
out_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8647 entries, 0 to 8646
Data columns (total 12 columns):
PackDate     8647 non-null datetime64[ns]
MotherSN     8647 non-null object
MB           8647 non-null object
NIBP         8647 non-null object
SPO2         8647 non-null object
GLU          8647 non-null object
ECG          8634 non-null object
NFC          8647 non-null object
INTERFACE    8647 non-null object
BATTERY      7695 non-null object
LCD          8498 non-null object
3G           4234 non-null object
dtypes: datetime64[ns](1), object(11)
memory usage: 810.7+ KB


,PackDate,MotherSN,MB,NIBP,SPO2,GLU,ECG,NFC,INTERFACE,BATTERY,LCD,3G
0,2018-03-05,10006244181W0053,000194181T0002A0,120024181N004JC0,110044181Q002NC0,04001417CC006SC0,SZGR201701416,090024181S005ZD0,15001417CL001CD0,4555107GB1B4722,XR-X097013AB801100432,A0061483253C
1,2018-03-05,1000624418210035,000194181W000WA0,120024181N002AC0,110044181Q0060C0,040014181N001KC0,SZGR201702255,090024181S001CD0,15001417C7008ND0,4555107GB1B0153,XR-X097013AB801101101,A00614830FF1
2,2018-03-05,1000624418210117,000194181W000HA0,120024181W002AC0,110044181X0014C0,040014181V00L1C0,SZGR201702985,090024181W002ZD0,15001417CL00AQD0,4555107GB1B0754,XR-X097013AB801101052,A006148308C6
3,2018-03-05,1000624418210123,000194181W0025A0,120024181W0047C0,110044181X00D2C0,040014181V002PC0,SZGR201609873,090024181W005QD0,15001417CL00B1D0,4555107GB1B0123,XR-X097013AB801100086,A00614830A12
4,2018-03-05,1000624418210138,000194181W004ZA0,120024181W006WC0,110044181X006JC0,040014181V0024C0,SZGR201702973,090024181W002TD0,15001417CL004DD0,4555107GB1B0732,XR-X097013AB801101020,A00614831F21


In [15]:
out_data.describe()

,PackDate,MotherSN,MB,NIBP,SPO2,GLU,ECG,NFC,INTERFACE,BATTERY,LCD,3G
count,8647,8647,8647,8647,8647,8647,8634,8647,8647,7695,8498,4234
unique,143,8647,8641,8642,8634,8637,8630,8632,8595,7646,8475,4234
top,2018-04-13 00:00:00,10006666187T0092,000226184B0265A0,20002618540274D0,110046184G0054C0,040016187X0101C0,JSGR201807850,090046184W0452A0,150026183Q0418A0,XH-X080H18A,XR-X097013AB801100019,A00328720E9E
freq,75,1,3,2,2,2,2,3,2,45,4,1
first,2018-03-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2018-09-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
out_data.to_excel("Product_Child_SN.xls")